# The map of Per Capita Income per Riding for 2015 elections

Legend:

* Red - Liberal Party of Canada

* Blue - Conservative Party of Canada

* Orange - New Democratic Party

* Light Blue - Bloc Québécois

* Green  - Green Party of Canada

The relative size of the circle corresponds to the per capita income for the Riding.

**N.B.** How to calculate per capita income?
    For example, in final_elected.csv, for FED 10001, the population is 81540, but 68487 electors; in income.csv, 'Total' is 72030. I guess, the amount of Income was calculated for all the taxpayers (72030), that included those who didn't have the right to vote. 72030 - 68487 = 3543 That many???

In [1]:
# Dependencies and initialization
import gmaps
import pandas as pd

# Google developer API key
from api_keys import g_key

# Data sources
geodata_csv = 'geodata.csv' # FED Coordinates (key: 'FED Id')
provinces_csv = 'provinces.csv' # Provinces Ids and coordinates (key: 'Province Id')
parties_csv = '../data/winning_parties_2015.csv' # List of winning parties (key: 'Custom Name')
results_csv = '../data/final_elected.csv' # List of election results by FED (keys: 'FED ID', 'Party Won')
income_csv = '../data/income.csv' # FED Income Data (key: 'FED ID')

# Conclusions

**No conclusions yet.**

In [2]:
# Read data
geodata_df = pd.read_csv(geodata_csv).loc[:, ['FED Id', 'FED Name', 'Latitude', 'Longitude']]
provinces_df = pd.read_csv(provinces_csv)
parties_df = pd.read_csv(parties_csv).loc[:, ['Id', 'Custom Name']]
results_df = pd.read_csv(results_csv).loc[:, ['FED ID', 'Party Won']]
income_df = pd.read_csv(income_csv, encoding='windows-1252').loc[:, ['Prov/Terr', 'FED ID', 'Total', 'Net Income']]
prov_income_df = income_df[income_df['Prov/Terr'] == 'TOTAL'].copy()
income_df = income_df[income_df['Prov/Terr'] != 'TOTAL']

In [3]:
# Some useful functions

# From the pd.Series s with numeric values calculate scale for each point
def make_scale_list(s, bin_count=10):
    interval_index = pd.interval_range(start=s.min() * 0.999, end=s.max(), periods=bin_count)
    return [1 + 1 * int(interval_index.get_loc(val)) for val in s]

# From pd.Series s with Party Ids make a list of party colors
def make_color_list(s):
    # Source: https://www.rapidtables.com/web/color/RGB_Color.html
    c_blue      = (51, 51, 255)
    c_lightblue = (51, 153, 255)
    c_green     = (0, 204, 0)
    c_orange    = (255, 165, 0)
    c_red       = (255, 0, 0)

    # Source: https://en.wikipedia.org/wiki/Political_colour
    party_colors = {
        'CPC': c_blue,
        'BC' : c_lightblue,
        'GRN': c_green,
        'NDP': c_orange,
        'LIB': c_red
    }

    return s.map(lambda id: party_colors[id]).tolist()

In [4]:
# Create data set for FEDs
income_df['Per Capita'] = income_df['Net Income'] / income_df['Total']

feds_df = pd.merge(geodata_df, results_df, how='left', left_on='FED Id', right_on='FED ID')
feds_df = pd.merge(feds_df, parties_df, how='left', left_on='Party Won', right_on='Custom Name')
feds_df = pd.merge(feds_df, income_df, how='left', left_on='FED Id', right_on='FED ID')

feds_df.drop(columns=['FED ID_x', 'FED ID_y', 'Custom Name', 'Prov/Terr'], inplace=True)
#df.drop(columns=['FED ID_x', 'FED ID_y', 'Custom Name', 'Prov/Terr', 'Total', 'Net Income'], inplace=True)

feds_df.rename(columns={'Id': 'Party Id'}, inplace=True)

#feds_df = feds_df[feds_df['Party Id'] == 'NDP'] # Filter for testing
feds_df.head()

,FED Id,FED Name,Latitude,Longitude,Party Won,Party Id,Total,Net Income,Per Capita
0,11001,Cardigan,46.251413,-62.652435,Liberal,LIB,29970,1.140059e+09,38040.006673
1,11002,Charlottetown,46.267987,-63.143870,Liberal,LIB,29650,1.098089e+09,37035.042159
2,24056,Pierrefonds--Dollard,45.491925,-73.855406,Liberal,LIB,86110,3.575200e+09,41518.987342
3,24057,Pontiac,46.760139,-76.594503,Liberal,LIB,88560,3.781101e+09,42695.359079
4,24058,Portneuf--Jacques-Cartier,47.137004,-71.836506,Conservative,CPC,92380,4.005662e+09,43360.705780


In [5]:
# Create dataset for Provinces
# Fixing prov_income_df: no prov ids, no names, only numbers :((

# The last line is Total for Canada. Don't need it.
if len(prov_income_df) > len(provinces_df): # Delete line only once
    prov_income_df = prov_income_df.iloc[0:-1, :]

# Fortunately, the data in prov_income_df is sorted by prov_id
# We can borrow the index from provinces_df
prov_income_df.set_index(provinces_df['Province Id'], inplace=True)

# Making resulting dataset good for display
provs_df = pd.merge(provinces_df, prov_income_df, how='left', left_on='Province Id', right_index=True)
provs_df.drop(columns=['Prov/Terr', 'FED ID'], inplace=True)
provs_df['Per Capita'] = provs_df['Net Income'] / provs_df['Total']
provs_df['Taxpayers'] = provs_df['Total'].map("{:,d}".format)
provs_df['Average Per Capita Income ($)'] = provs_df['Per Capita'].map("{:,.0f}".format)
provs_df

,Province Id,Province Name,Latitude,Longitude,Total,Net Income,Per Capita,Taxpayers,Average Per Capita Income ($)
0,10,Newfoundland and Labrador,51.039105,-56.715888,433100,1.774340e+10,40968.372200,"433,100","40,968"
1,11,Prince Edward Island,46.476074,-63.312972,117810,4.340412e+09,36842.475172,"117,810","36,842"
2,12,Nova Scotia,45.262660,-62.808875,753400,2.877862e+10,38198.331564,"753,400","38,198"
3,13,New Brunswick,46.475575,-66.013411,615360,2.246257e+10,36503.141251,"615,360","36,503"
4,24,Quebec,49.696473,-71.909004,6573000,2.550850e+11,38808.002434,"6,573,000","38,808"
5,35,Ontario,47.527625,-83.360554,10679920,4.759570e+11,44565.595997,"10,679,920","44,566"
6,46,Manitoba,52.434901,-97.963706,985170,3.868995e+10,39272.361115,"985,170","39,272"
7,47,Saskatchewan,53.068231,-106.041076,848490,3.771990e+10,44455.331235,"848,490","44,455"
8,48,Alberta,53.590719,-114.652379,3099120,1.636450e+11,52803.699115,"3,099,120","52,804"
9,59,British Columbia,52.476869,-123.534440,3790190,1.660160e+11,43801.498078,"3,790,190","43,801"


In [6]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '675px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
center=(43.6551165, -79.3869946) # Downtown Toronto

fig = gmaps.figure(layout=figure_layout, center=center, zoom_level=8)

In [7]:
# The FEDs layer with symbols

# FEDs central points
locations = feds_df[['Latitude', 'Longitude']]
hover_text = feds_df['FED Name'].tolist()
fill_color = make_color_list(feds_df['Party Id'])
stroke_color = fill_color
scale = make_scale_list(feds_df.iloc[:, -1], bin_count=15)

data_layer = gmaps.symbol_layer(
    locations, # hover_text="hover_text", - doesn't work :(
    fill_color=fill_color, stroke_color=stroke_color,
    scale=scale
)

# Add the layer to the map
fig.add_layer(data_layer)

In [8]:
# The Provinces layer with markers and infoboxes

# Province center points
center_points = provs_df[['Latitude', 'Longitude']]

# Using the template add info boxes
info_box_template = """
<dl>
<dt>Province</dt><dd>{Province Name}</dd>
<dt>Taxpayers</dt><dd>{Taxpayers}</dd>
<dt>Average Per Capita Income ($)</dt><dd>{Average Per Capita Income ($)}</dd>
</dl>
"""
info_boxes = [info_box_template.format(**row) for index, row in provs_df.iterrows()]

markers = gmaps.marker_layer(center_points, info_box_content=info_boxes)

# Add the layer to the map
fig.add_layer(markers)

In [9]:
# Display
fig

Figure(layout=FigureLayout(border='1px solid black', height='675px', margin='0 auto 0 auto', padding='1px', wi…